In [2]:
import pandas as pd
import numpy as np
# STEP 1: Load the dataset
# Equivalent to: Connecting source data in Tableau Prep Builder
# ----------------------------
df = pd.read_csv('/Users/maneshreddy/Downloads/seda_commzone_long_gcs_5.0.csv')
# ----------------------------
# STEP 2: Clean column names
# Equivalent to: Renaming fields for consistency
# ----------------------------
df.columns = df.columns.str.strip().str.lower()
# ----------------------------
# STEP 3: Handle missing values
# Equivalent to: Cleaning null/empty fields in Tableau Prep
# ----------------------------
df = df.replace(r'^\s*$', np.nan, regex=True)  # Replace empty strings with NaN
# ----------------------------
# STEP 4: Convert data types
# Equivalent to: Changing field data types in Tableau Prep
# ----------------------------
numeric_cols = [col for col in df.columns if col.startswith(('gcs_', 'tot_')) or col in ['grade', 'year']]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
# ----------------------------
# STEP 5: Reshape data to long format
# Equivalent to: Pivoting (melt/unpivot) in Tableau Prep
# ----------------------------
demographics = ['all', 'asn', 'blk', 'ecd', 'fem', 'hsp', 'mal', 'mfg', 'nam', 
                'nec', 'neg', 'wag', 'wbg', 'whg', 'wht', 'wng']
metrics = ['gcs_mn', 'gcs_mn_se', 'tot_asmt']
melted_dfs = []
for demo in demographics:
    demo_cols = [f"{metric}_{demo}" for metric in metrics if f"{metric}_{demo}" in df.columns]
    if not demo_cols:
        continue 
    demo_df = df[['sedacz', 'subject', 'grade', 'year'] + demo_cols].copy()
    demo_df['demographic'] = demo
    # Rename columns to a common structure (like Tableau Prep field renaming)
    col_mapping = {col: col.replace(f"_{demo}", "") for col in demo_cols}
    demo_df.rename(columns=col_mapping, inplace=True)

    melted_dfs.append(demo_df)
# Combine all long-format tables
long_df = pd.concat(melted_dfs, ignore_index=True)
# ----------------------------
# STEP 6: Add calculated fields
# Equivalent to: Calculated Fields in Tableau Prep Builder
# ----------------------------
demographic_labels = {
    'all': 'All Students', 'asn': 'Asian', 'blk': 'Black',
    'ecd': 'Economically Disadvantaged', 'fem': 'Female', 'hsp': 'Hispanic',
    'mal': 'Male', 'mfg': 'Minority Focus Group', 'nam': 'Native American',
    'nec': 'Non-Economically Disadvantaged', 'neg': 'Non-English Speaking',
    'wag': 'White (All Grades)', 'wbg': 'White (Below Grade)', 'whg': 'White (High Grade)',
    'wht': 'White', 'wng': 'White (Non-Graded)'
}
long_df['demographic_label'] = long_df['demographic'].map(demographic_labels)

# Create subject-grade combo field
long_df['subject_grade'] = long_df['subject'].str.upper() + ' Grade ' + long_df['grade'].astype(str)

# Create year range bins for timeline grouping
long_df['year_range'] = pd.cut(long_df['year'], 
                              bins=[2008, 2012, 2016, 2020],
                              labels=['2009-2012', '2013-2016', '2017-2019'])
# ----------------------------
# STEP 7: Export Cleaned Outputs
# Equivalent to: Output step in Tableau Prep
# ----------------------------
long_df.to_csv('seda_processed.csv', index=False)            # Long format for dashboard visuals
df.to_csv('seda_wide_processed.csv', index=False)            # Wide format for alternate use

print("✅ Preprocessing complete.")
print("- Long format saved as: seda_processed.csv")
print("- Wide format saved as: seda_wide_processed.csv")


Preprocessing complete. Data saved in two formats:
- Long format (better for demographic analysis): seda_processed.csv
- Wide format (original structure): seda_wide_processed.csv
